Importando Bibliotecas

In [ ]:
import requests
from urllib.request import urlopen, urlretrieve,Request
import pandas as pd
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

Definindo os intervalos

In [ ]:
years = list(range(2013,2023))
months = list(['january', 'february', 'march', 'april', 'may', 'june', 'july',
          'august', 'septe               mber', 'october', 'november', 'december'])

## Tabela de Jogadores

### Coleta de Dados

Salvando HTML com dados de jogo de cada jogador

In [ ]:
url_players = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
for year in years:
    response = urlopen(url_players.format(year))
    html = response.read()
    with open ("players/{}.html".format(year),"w+", encoding='utf-8') as f:
        f.write(html.decode('utf-8'))

Transformando HTML em um DataFrame

In [ ]:
dfs = []
for year in years:
    with open ("players/{}.html".format(year),"r",encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page,"html.parser")
    soup.find('tr',class_="thead").decompose
    players_profile = soup.find_all(id="per_game_stats")
    players_profile = soup.select('tbody tr td[data-stat="player"] a')
    players_table = soup.find_all(id="per_game_stats")
    players_df = pd.read_html(str(players_table))[0]
    players_df.drop_duplicates(keep = False, inplace = True)
    players_df['Season']=year
    dfs.append(players_df)
    players = pd.concat(dfs)
players.to_csv("players.csv")

###  Tratamento de Dados

Analisando o DataFrame

In [ ]:
players = pd.read_csv("players.csv")
players

Modelagem dos dados

In [ ]:
#Limpeza das variáveis que não serão utilizadas
del players["Unnamed: 0"]
del players["Rk"]

# remover "*" dos nomes de jogadores
players["Player"] = players["Player"].str.replace("*","", regex=False)

# ajuste da abreviação do time Brooklyn Nets de "BRK" para "BKN"
players["Tm"] = players["Tm"].str.replace("BRK","BKN", regex=False)

In [ ]:
# Quando um jogador muda de time durante a temporada, a tabela cria uma nova linha com as estatísticas do novo time. 
# E também, cria mais uma linha com as estatísticas totais do jogador por temporada. De modo que temos os dados do jogador por time e também o somatório desses dados na temporada.
# Então, para fins de padronização da tabela, os dados mantidos serão os totais, atribuidos ao time onde o jogador encerrou a temporada.
players[players["Player"] == "James Harden"][players["Season"]>2020]

In [ ]:
# Função para identificar se há mais de uma linha para cada jogador e retornar os valores de "TOT", em caso de mais de uma ocorrência, substituindo, após, o nome do time de "TOT" para o último que surgir  
def team_tot(df):
    if df.shape[0]==1:
        return df
    else:
        row = df[df["Tm"]=="TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row
# Aplicar função no groupby
players = players.groupby(["Player","Season"]).apply(team_tot)

In [ ]:
#removendo index para manter o formato da tabela original
players.index = players.index.droplevel()
players.index = players.index.droplevel()
players

## Tabela MVP

### Coleta de Dados

Salvando HTML

In [ ]:
url_mvp = "https://www.basketball-reference.com/awards/awards_{}.html"
for year in years:
    response = urlopen(url_mvp.format(year))
    html = response.read()
    with open ("mvp/{}.html".format(year),"w+", encoding='utf-8') as f:
        f.write(html.decode('utf-8'))

Transformando HTML em um DataFrame

In [ ]:
dfs = []
for year in years:
    with open ("mvp/{}.html".format(year),"r",encoding='utf-8') as f:
        page = f.read()
        
    soup = BeautifulSoup(page,"html.parser")
    soup.find('tr', class_="over_header").decompose()
    link = soup.find('td')
    link = link.find('a')
    link2 = link.get('href')
    mvp_table = soup.find_all(id="mvp")
    mvp_df = pd.read_html(str(mvp_table))[0]
    mvp_df["Season"] = year
    dfs.append(mvp_df)
    mvps = pd.concat(dfs)
    mvps.to_csv("mvps.csv")

###  Tratamento de Dados

Analisando o DataFrame

In [ ]:
mvps = pd.read_csv("mvps.csv")

Modelagem dos dados

In [ ]:
#Acrescentar variável informando quem foi o MVP da temporada
mvps["MVP"]=""
mvps["MVP"].mask(mvps["Rank"] == "1", "MVP", inplace=True)
mvps["MVP"].mask(mvps["Rank"] !="1", "Not MVP", inplace=True)

In [ ]:
#Limpeza das variáveis que não serão utilizadas
mvps = mvps[["Player","Season","Pts Won",'Share',"MVP"]]
mvps

## Tabela de Times

### Coleta de Dados

Salvando HTML com dados de cada time

In [ ]:
url_teams = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"
for year in years:
    response = urlopen(url_teams.format(year))
    html = response.read()
    with open ("teams/{}.html".format(year),"w+", encoding='utf-8') as f:
        f.write(html.decode('utf-8'))

Transformando HTML em um DataFrame

In [ ]:
dfs=[]
for year in years:
    with open ("teams/{}.html".format(year),"r",encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page,"html.parser")
    soup.find('tr',class_="thead").decompose()
    
    e_table = soup.find_all(id="div_divs_standings_E")[0]
    e_df = pd.read_html(str(e_table))[0]
    e_df.rename(columns = {'Eastern Conference':'Team'},inplace=True)
    e_df['Season']=year
    e_df['Conference']='Eastern'
    e_df.drop(e_df.loc[e_df['Team'].str.contains("Division")].index, inplace=True)
    dfs.append(e_df)

    w_table = soup.find_all(id="div_divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df.rename(columns = {'Western Conference':'Team'},inplace=True)
    w_df['Season']=year
    w_df['Conference']='Western'
    w_df.drop(w_df.loc[w_df['Team'].str.contains("Division")].index, inplace=True)
    dfs.append(w_df)
teams = pd.concat(dfs)   
teams.to_csv("teams.csv")

###  Tratamento de Dados

Analisando o DataFrame

In [ ]:
teams = pd.read_csv('teams.csv')
teams

Modelagem dos dados

In [ ]:
# deletar variáveis que não serão utilizadas
del teams["GB"]
del teams["PS/G"]
del teams["PA/G"]
del teams["SRS"]

In [ ]:
# Criação de variável indicando que o time foi para o torneio de Playoffs
teams["Playoffs"] = teams['Team'].str[-1]
teams['Playoffs'].mask(teams['Playoffs'] != "*", "Não Classificado", inplace=True)
teams['Playoffs'].mask(teams['Playoffs'] == "*","Classificado", inplace=True)
teams["Team"]= teams["Team"].str.replace("*","", regex=True)

In [ ]:
# Dicionário com abreviações e nomes de time
teamname = {}
with open ("teamnames.csv") as f:
    lines = f.readlines()
    for line in lines[1:]:
        line = line.replace("\n","")
        line = line.split(";")
        teamname[line[1]]=line[0]

In [ ]:
#Aplicando dicionário para cruzar tabela de times com as demais
teams["Tm"] = teams["Team"].map(teamname)
teams

## Consolidação dos Dados Coletados

In [ ]:
#Players Table + MVP Table
players_tb = players.merge(mvps, how="outer", on = ['Player','Season'])

In [ ]:
#Players + MVP + Teams Table
nba_stats = players_tb.merge(teams, how = 'outer', on = ["Tm","Season"])

In [ ]:
# Nem todos os jogadores foram votados para MVP, portanto, é necessário preencher os registros nulos a fim de evitar erros de cálculo
nba_stats[["Pts Won","Share","MVP"]] = nba_stats[["Pts Won","Share","MVP"]].fillna(0)

In [ ]:
#Transformar os dados em formato numérico
nba_stats = nba_stats.apply(pd.to_numeric, errors="ignore")

In [ ]:
del nba_stats["Unnamed: 0"]
del nba_stats["Conference"]
nba_stats.to_csv("NBA_Stats.csv")

## Dados Consolidados

In [ ]:
nba_stats = pd.read_csv("NBA_Stats.csv")
del nba_stats["Unnamed: 0"]
nba_stats

In [ ]:
## Filtrando jogadores que jogaram mais de 10 jogos na temporada
nba_stats = nba_stats[nba_stats["G"]>10]
nba_stats

In [ ]:
##Preencher campos em branco com zero
nba_stats = nba_stats.fillna(0)

In [ ]:
## Acrescentando Média de desempenho por idade agrupado
avg_age = nba_stats.groupby(['Age']).mean()
nba_stats = nba_stats.merge(avg_age['MP'], how="outer", on = ['Age'])
nba_stats = nba_stats.merge(avg_age['PTS'], how="outer", on = ['Age'])
nba_stats.rename(columns={'MP_y':'MP_Avg'}, inplace = True)
nba_stats.rename(columns={'MP_x':'MP'}, inplace = True)
nba_stats.rename(columns={'PTS_x':'PTS'}, inplace = True)
nba_stats.rename(columns={'PTS_y':'PTS_Avg'}, inplace = True)
nba_stats[["MP_Avg"]]=nba_stats[["MP_Avg"]].fillna(0)
nba_stats[["PTS_Avg"]]=nba_stats[["PTS_Avg"]].fillna(0)
nba_stats

In [ ]:
nba_stats.describe().transpose()

## Visualização dos dados 

In [ ]:
# Barras com a idade dos jogadores ao longo das temporadas
sns.barplot(data=nba_stats,x="Season",y="Age",hue="Age")
plt.legend().remove()
plt.show()

In [ ]:
# Boxplot com a idade dos jogadores ao longo das temporadas
sns.boxplot(data=nba_stats, y = 'Age',  x = "Season")

In [ ]:
## Análise descritiva da variável idade em 2016
nba_idade = nba_stats[nba_stats['Season']==2016]['Age'].describe()
del nba_idade['count']
nba_idade

In [ ]:
# Boxplot com a idade e pontos
plt.figure(figsize=(20,5))
sns.boxplot(data=nba_stats, x='Age', y='PTS')

In [ ]:
# Dispersão com a disposição da média de minutos jogados em função da idade
fig = px.scatter(nba_stats, y="MP_Avg", x ="Age", 
width=800, height=600)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
# Histograma com principais estatístivas de jogo em função da idade
fig, ax =plt.subplots(3,3,constrained_layout = True)
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['FG'], ax=ax[0,0])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['2P'], ax=ax[0,1])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['3P'], ax=ax[0,2])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['FT'], ax=ax[1,0])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['TRB'], ax=ax[1,1])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['AST'], ax=ax[1,2])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['STL'], ax=ax[2,0])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['BLK'], ax=ax[2,1])
sns.histplot(data=nba_stats, x = nba_stats['Age'],y=nba_stats['TOV'], ax=ax[2,2])
plt.figure(figsize=(12, 12))
plt.show()

In [ ]:
# Heatmap com correlação
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(nba_stats.corr()[['Age']].sort_values(by='Age', ascending=False), vmin=-1, vmax=1, annot=True, cmap='RdBu')

In [ ]:
#Correlação de jogadores mais novos
plt.figure(figsize=(5, 8))
plt.title("Jogadores com 26 anos ou menos", fontsize =11)
heatmap = sns.heatmap(nba_stats[nba_stats['Age']<27].corr()[['Age']].sort_values(by='Age', ascending=False), vmin=-1, vmax=1, annot=True, cmap='PiYG')

In [ ]:
#Correlação de jogadores mais velhos
plt.figure(figsize=(5, 8))
plt.title("Jogadores com mais de 26 anos", fontsize =11)
heatmap = sns.heatmap(nba_stats[nba_stats['Age']>26].corr()[['Age']].sort_values(by='Age', ascending=False), vmin=-1, vmax=1, annot=True, cmap='RdGy')